In [17]:

from sklearn.model_selection import ShuffleSplit
import xgboost as xgb
import sys
sys.path.append("../") 
from dataset.nhanes import NHANESDataset
import numpy as np
import pandas as pd

# NHANES

In [18]:
data = NHANESDataset()

/mnt/c/Users/rizki/Master Files/ThesisUMCG/Survival-Tree-Analysis/src/notebook/../dataset/nhanes.py:68: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.data.replace({False: 0, True: 1}, inplace=True)


In [19]:
d = data.get_data()
d

array([[  0.        ,  51.        ,   3.        , ..., 110.        ,
         40.        ,  25.40680287],
       [  0.        ,  41.        ,   2.        , ..., 136.        ,
         54.        ,  24.58833109],
       [  1.        ,  31.        ,   2.        , ..., 110.        ,
         24.        ,  23.75650236],
       ...,
       [  1.        ,  47.        ,   2.        , ..., 108.        ,
         28.        ,  17.9151988 ],
       [  1.        ,  61.        ,   8.        , ..., 180.        ,
         68.        ,  27.33162171],
       [  0.        ,  29.        ,   3.        , ..., 152.        ,
         60.        ,  33.16143518]])

In [20]:
X_train, X_test, y_train, y_test = data.get_train_test_xgboost()

In [21]:
y_train

,Survival_label_lower_bound,Survival_label_upper_bound
7556,13.750000,13.750000
819,11.500000,11.500000
12960,17.833333,inf
4609,12.083333,12.083333
5028,20.083333,inf
...,...,...
5191,2.500000,2.500000
13418,0.833333,0.833333
5390,20.000000,inf
860,6.250000,6.250000


In [22]:
dtrain = xgb.DMatrix(X_train)
dtrain.set_float_info('label_lower_bound', y_train['Survival_label_lower_bound'])
dtrain.set_float_info('label_upper_bound', y_train['Survival_label_upper_bound'])

In [23]:
dvalid = xgb.DMatrix(X_test)
dvalid.set_float_info('label_lower_bound', y_test['Survival_label_lower_bound'])
dvalid.set_float_info('label_upper_bound', y_test['Survival_label_upper_bound'])

In [24]:
params = {'verbosity': 0,
          'objective': 'survival:aft',
          'eval_metric': 'aft-nloglik',
          'tree_method': 'hist',
          'learning_rate': 0.05,
          'aft_loss_distribution': 'normal',
          'aft_loss_distribution_scale': 1.2,
          'max_depth': 6,
          'lambda': 0.01,
          'alpha': 0.02}

bst = xgb.train(params, dtrain, num_boost_round=1000,
                evals=[(dtrain, 'train')],
                early_stopping_rounds=10)

[0]	train-aft-nloglik:6.07962
[1]	train-aft-nloglik:5.65744
[2]	train-aft-nloglik:5.27431
[3]	train-aft-nloglik:4.92694
[4]	train-aft-nloglik:4.61164
[5]	train-aft-nloglik:4.32516
[6]	train-aft-nloglik:4.06503
[7]	train-aft-nloglik:3.82881
[8]	train-aft-nloglik:3.61405
[9]	train-aft-nloglik:3.41886
[10]	train-aft-nloglik:3.24123
[11]	train-aft-nloglik:3.07964
[12]	train-aft-nloglik:2.93247
[13]	train-aft-nloglik:2.79846
[14]	train-aft-nloglik:2.67632
[15]	train-aft-nloglik:2.56493
[16]	train-aft-nloglik:2.46338
[17]	train-aft-nloglik:2.37075
[18]	train-aft-nloglik:2.28613
[19]	train-aft-nloglik:2.20887
[20]	train-aft-nloglik:2.13837
[21]	train-aft-nloglik:2.07403
[22]	train-aft-nloglik:2.01510
[23]	train-aft-nloglik:1.96130
[24]	train-aft-nloglik:1.91208
[25]	train-aft-nloglik:1.86701
[26]	train-aft-nloglik:1.82566
[27]	train-aft-nloglik:1.78795
[28]	train-aft-nloglik:1.75329
[29]	train-aft-nloglik:1.72151
[30]	train-aft-nloglik:1.69243
[31]	train-aft-nloglik:1.66584
[32]	train-aft-nlo

In [25]:
times_pred = bst.predict(dvalid, output_margin=True)
times_pred

array([5.805531 , 5.164848 , 5.3984466, ..., 2.2276459, 4.1012335,
       5.1468596], dtype=float32)

0.8010898871836367